In [4]:
import warnings
warnings.filterwarnings("ignore")
!pip install -r requirements.txt -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import sys
import os 
from tqdm import tqdm

from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

In [6]:
df= pd.read_csv("train.csv")
df["question1"] = df["question1"].apply(lambda x: str(x))
df["question2"] = df["question2"].apply(lambda x: str(x))

In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [14]:
## Merging texts of questions
questions = list(df['question1'] + df['question2'])

## Finding tfidf of questions
tfidf = TfidfVectorizer(lowercase=False)
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [19]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 587.7 MB 9.0 kB/s eta 0:00:016    |██████▉                         | 126.4 MB 490 kB/s eta 0:15:41
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [24]:
nlp = spacy.load('en_core_web_lg')

vecs1 = []
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1)
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)]) 

    for word1 in doc1:
        vec1 = word1.vector

        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        mean_vec1 += vec1*idf

    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)

df["q1_feats_m"] = list(vecs1)



  6%|▌         | 23629/404290 [08:34<2:19:46, 45.39it/s]

In [ ]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2)
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)]) 

    for word2 in doc2:
        vec2 = word2.vector

        try:
            idf = word2tfidf[str(word2)]
        except:
            idf = 0
        mean_vec2 += vec2*idf

    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)

df["q2_feats_m"] = list(vecs2)